# MT5 Historical Data Fetcher

This notebook fetches historical data from MetaTrader 5 in chunks of 1000 bars and combines them into a pandas DataFrame.

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import datetime
import pytz
import time
import numpy as np
import matplotlib.pyplot as plt

# Set display options for pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)

## Initialize MT5 Connection

In [2]:
# Initialize MT5
if not mt5.initialize():
    print(f"initialize() failed, error code = {mt5.last_error()}")
    quit()

# Display MT5 version
print(f"MetaTrader5 package version: {mt5.__version__}")
print(f"MetaTrader5 terminal version: {mt5.version()}")

MetaTrader5 package version: 5.0.4424
MetaTrader5 terminal version: (500, 4755, '13 Dec 2024')


## Define Parameters

In [10]:
# Set parameters
symbol = "XAUUSD"  # Symbol to fetch
timeframe = mt5.TIMEFRAME_M1  # Timeframe (H1 = 1 hour)
max_bars = 100  # Number of bars to fetch per request

# Set timezone to UTC
timezone = pytz.timezone("Etc/UTC")

# Set start date (adjust as needed)
start_date = datetime.datetime(2023, 1, 1, tzinfo=timezone)
end_date = datetime.datetime(2023, 12, 31, tzinfo=timezone)

## Fetch Historical Data in Chunks

In [11]:
def fetch_historical_data(symbol, timeframe, start_date, end_date, max_bars=1000):
    """Fetch historical data in chunks and combine into a DataFrame"""
    all_data = []
    current_date = start_date
    
    while current_date < end_date:
        print(f"Fetching data from {current_date.isoformat()}")
        
        # Fetch data
        rates = mt5.copy_rates_from(symbol, timeframe, current_date, max_bars)
        print(type(current_date))
        print(rates)
        
        if rates is None or len(rates) == 0:
            print(f"No data returned for {current_date.isoformat()}")
            break
            
        # Convert to DataFrame
        df_chunk = pd.DataFrame(rates)
        
        # Convert time in seconds into datetime format
        df_chunk['time'] = pd.to_datetime(df_chunk['time'], unit='s')
        
        # Filter out data beyond end_date
        df_chunk = df_chunk[df_chunk['time'] <= end_date]
        
        if len(df_chunk) == 0:
            break
            
        # Add to our collection
        all_data.append(df_chunk)
        
        # Update current_date to the last timestamp + 1 timeframe unit
        last_time = df_chunk['time'].iloc[-1]
        
        # Add appropriate time delta based on timeframe
        if timeframe == mt5.TIMEFRAME_M1:
            current_date = last_time + datetime.timedelta(minutes=1)
        elif timeframe == mt5.TIMEFRAME_M5:
            current_date = last_time + datetime.timedelta(minutes=5)
        elif timeframe == mt5.TIMEFRAME_M15:
            current_date = last_time + datetime.timedelta(minutes=15)
        elif timeframe == mt5.TIMEFRAME_M30:
            current_date = last_time + datetime.timedelta(minutes=30)
        elif timeframe == mt5.TIMEFRAME_H1:
            current_date = last_time + datetime.timedelta(hours=1)
        elif timeframe == mt5.TIMEFRAME_H4:
            current_date = last_time + datetime.timedelta(hours=4)
        elif timeframe == mt5.TIMEFRAME_D1:
            current_date = last_time + datetime.timedelta(days=1)
        
        # If we got fewer than max_bars, there's no more data
        if len(rates) < max_bars:
            break
            
        # Add a small delay to avoid overwhelming the MT5 API
        time.sleep(0.1)
    
    # Combine all chunks into a single DataFrame
    if all_data:
        df = pd.concat(all_data, ignore_index=True)
        return df
    else:
        return pd.DataFrame()

In [12]:
# Fetch the data
df = fetch_historical_data(symbol, timeframe, start_date, end_date, max_bars)

# Display the first few rows
print(f"Total rows: {len(df)}")
df.head()

Fetching data from 2023-01-01T00:00:00+00:00
<class 'datetime.datetime'>
None
No data returned for 2023-01-01T00:00:00+00:00
Total rows: 0


""


## Basic Data Analysis

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

In [ ]:
# Plot the closing prices
plt.figure(figsize=(14, 7))
plt.plot(df['time'], df['close'])
plt.title(f'{symbol} Closing Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.show()

In [ ]:
# Calculate daily returns
df['returns'] = df['close'].pct_change()

# Plot the returns
plt.figure(figsize=(14, 7))
plt.plot(df['time'], df['returns'])
plt.title(f'{symbol} Returns')
plt.xlabel('Date')
plt.ylabel('Returns')
plt.grid(True)
plt.show()

In [ ]:
# Save the data to CSV
output_file = f"{symbol}_{timeframe}_data.csv"
df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")

In [ ]:
# Shutdown MT5 connection
mt5.shutdown()